# JEMSSWrapper Basic Test Notebook
This notebook tests loading the JEMSSWrapper and accessing some utilities.

In [1]:
# Activate local project
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.resolve()

  Activating project at `~/Projects/TFG/ExperimentFramework/JEMSSWrapper.jl`
  No Changes to `~/Projects/TFG/ExperimentFramework/JEMSSWrapper.jl/Project.toml`
  No Changes to `~/Projects/TFG/ExperimentFramework/JEMSSWrapper.jl/Manifest.toml`


In [2]:
# Load the wrapper
using JEMSSWrapper

In [3]:
# Utility: get JEMSS info
function get_jemss_info()
    jemss_info = JEMSSWrapper.get_jemss_info()
    return (
        path = jemss_info.path,
        exports = jemss_info.exports
    )
end

path, exports = get_jemss_info();

## Placeholder Simulation
Replace this section with actual JEMSS simulation code when available.

In [6]:
println("=== Running Basic Simulation Test ===")
initSim = JEMSSWrapper.jemss.initSim
simulate! = JEMSSWrapper.jemss.simulate!

config_path = joinpath(path, "example", "input", "sim_config.xml")
sim = initSim(config_path)
simulate!(sim)
println("✓ Simulation completed successfully")
println("✓ Wrapper setup completed successfully")
println("✓ Ready for JEMSS simulation integration")

=== Running Basic Simulation Test ===
✓ Simulation completed successfully
✓ Wrapper setup completed successfully
✓ Ready for JEMSS simulation integration
